<a href="https://colab.research.google.com/github/lionelfragniere/SSD_EORE/blob/master/SSD_EORE_DRAFT_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>